# Scaling to large datasets

Pandas provides data structures for **in-memory analytics**, which makes using pandas to analyze datasets that are larger than memory datasets somewhat tricky.
Even datasets that are a sizable fraction of memory become unwieldy, as some pandas operations need to make intermediate copies.

The document [Scaling to large datasets – Pandas User Guide](https://pandas.pydata.org/docs/user_guide/scale.html) provides a few recommendations for scaling your analysis to larger datasets. It’s a complement to Enhancing performance, which focuses on speeding up analysis for datasets that fit in memory.

The documentation shows a few strategies for scaling to larger datasets:

- Load less data
- Use efficient datatypes
- **Use chunking**
- Use Other Libraries

In this notebook, we focus on **chunking** and present a more advanced example than the one shown in the official documentation. Here, we demonstrate how to process data in smaller pieces that fit into memory and perform chunk aggregation for a slightly more complex data processing task then the one shown in the documentation.


## File chunking with TextFileReader 

In [26]:
import pandas as pd
from pathlib import Path
from tqdm import tqdm

ds_100k_path = Path('.data') / '100k.csv'

In [27]:
tfr  = pd.read_csv(ds_100k_path, chunksize=100) 

type(tfr)

pandas.io.parsers.readers.TextFileReader

In [30]:
for i, chunk in enumerate(tfr):
    chunk.info()
    break

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 200 to 299
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Region          100 non-null    object 
 1   Country         100 non-null    object 
 2   Item Type       100 non-null    object 
 3   Sales Channel   100 non-null    object 
 4   Order Priority  100 non-null    object 
 5   Order Date      100 non-null    object 
 6   Order ID        100 non-null    int64  
 7   Ship Date       100 non-null    object 
 8   Units Sold      100 non-null    int64  
 9   Unit Price      100 non-null    float64
 10  Unit Cost       100 non-null    float64
 11  Total Revenue   100 non-null    float64
 12  Total Cost      100 non-null    float64
 13  Total Profit    100 non-null    float64
dtypes: float64(5), int64(2), object(7)
memory usage: 11.1+ KB


## Usage example: calculating the mean value of a column

Let us assume we would like to calculate the mean value of some column.

In the classic approach, one would use:

In [43]:
df = pd.read_csv(ds_100k_path)

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Region          100000 non-null  object 
 1   Country         100000 non-null  object 
 2   Item Type       100000 non-null  object 
 3   Sales Channel   100000 non-null  object 
 4   Order Priority  100000 non-null  object 
 5   Order Date      100000 non-null  object 
 6   Order ID        100000 non-null  int64  
 7   Ship Date       100000 non-null  object 
 8   Units Sold      100000 non-null  int64  
 9   Unit Price      100000 non-null  float64
 10  Unit Cost       100000 non-null  float64
 11  Total Revenue   100000 non-null  float64
 12  Total Cost      100000 non-null  float64
 13  Total Profit    100000 non-null  float64
dtypes: float64(5), int64(2), object(7)
memory usage: 10.7+ MB


Please note the memory usage of both DataFrames (last line in the load output).

In [33]:
s_reference = df.groupby('Region')['Unit Price'].mean()

s_reference

Region
Asia                                 265.792398
Australia and Oceania                262.774268
Central America and the Caribbean    268.143213
Europe                               266.845405
Middle East and North Africa         266.141459
North America                        270.677000
Sub-Saharan Africa                   267.651037
Name: Unit Price, dtype: float64

However, if the dataset is too big and we cannot load it to the memory, 
we may use the cunks.

In the example provided in the documentation, the goal is to calculate the value counts of a column.
In such case, the final value counts can be obtained by summing up the value counts of each chunk.
This procedure is called **chunk aggregation**.

Coming back to out example of calculating the mean value:

### **Question**: how to calculate the mean of all values while only having parts of the data.

Answer: Mathematics B)

We need to find (Google, GPT, Gemini, etc.) how to get the average of the whole group while having the averages of the subgroups:

> The weighted average of subgroup averages is mathematically identical to the average of the whole group.

So, we take the formula for the weighted average:

$$\text{Weighted Average} = \frac{\sum (\text{frag\_avg\_grade} \times \text{frag\_count})}{\sum \text{frag\_count}}$$

and implement the whole idea.

**Note**: chunk aggregation is usually the hard part of the whole chunk computation – you need to figure out how to combine the results from each chunk to get the final result.


In [42]:
# note: re-create the TextFileReader it was used up above
tfr  = pd.read_csv(
    ds_100k_path,
    chunksize=100
)

frag_stats = []
for chunk in tqdm(tfr, total=100e3/100, desc='Processing chunks', unit='chunk'):
    
    # for the fragment (subgroup/chunk/etc.)...
    df = chunk if type(chunk) == pd.DataFrame else chunk.to_frame() # series handling 
    df_frag_stats = df.groupby('Region').agg(
        frag_avg_grade=('Unit Price', 'mean'),   # ... calculate the mean ...
        frag_count=('Unit Price', 'count') 	# ... and the count (will be used as the weight).
    )
    frag_stats.append(df_frag_stats)  # Store only this aggregated data.

Processing chunks: 100%|██████████| 1000/1000.0 [00:02<00:00, 333.97chunk/s]


Concat it to one df

In [36]:
df_stats = pd.concat(frag_stats)

And then apply the mathematics

In [37]:
df_stats['prod'] = df_stats['frag_avg_grade'] * df_stats['frag_count']

result = df_stats.groupby(level=0)[['prod', 'frag_count']].sum()

result

,prod,frag_count
Region,,
Asia,3866482.02,14547
Australia and Oceania,2131887.64,8113
Central America and the Caribbean,2877444.82,10731
Europe,6905158.55,25877
Middle East and North Africa,3348059.55,12580
North America,577354.04,2133
Sub-Saharan Africa,6964012.32,26019


In [38]:
result['weighted_avg'] = result['prod'] / result['frag_count']
print(result[['weighted_avg']])

                                   weighted_avg
Region                                         
Asia                                 265.792398
Australia and Oceania                262.774268
Central America and the Caribbean    268.143213
Europe                               266.845405
Middle East and North Africa         266.141459
North America                        270.677000
Sub-Saharan Africa                   267.651037


In [39]:
assert(result['weighted_avg'].compare(s_reference).shape[0] == 0), "Results differ!"

As we can see, the results are identical to the non-chunked approach.

# Summary

Chunking is a technique for processing large datasets by splitting them into smaller, manageable pieces ("chunks") that fit into memory. Instead of loading the entire dataset at once, you read and process each chunk sequentially.

**Why use chunking?**
- Enables analysis of datasets larger than available memory.
- Reduces memory usage and risk of crashes.
- Allows streaming and incremental processing.

**Chunk aggregation** refers to combining results from each chunk to produce a final result (e.g., summing value counts, calculating weighted averages).

**Advantages:**
- Scales to very large datasets.
- Efficient memory usage.
- Can parallelize or distribute processing.

**Disadvantages:**
- Requires careful aggregation logic (e.g., weighted averages, merging counts).
- Some operations (sorting, global ranking) are harder or less efficient.
- More complex code compared to simple in-memory operations.

Chunking is essential for big data workflows, but chunk aggregation is often the challenging part, as you must design how to combine partial results correctly.
